In [11]:
import praw
import os
import datetime as dt
import time
import pandas as pd
from psaw import PushshiftAPI

In [12]:
# retrieving info w environment variables
USERNAME = os.environ.get('REDDIT_NLP_USERNAME')
PASSWORD = os.environ.get('REDDIT_NLP_PASSWORD')
CLIENT_ID = os.environ.get('REDDIT_NLP_CLIENT_ID')
CLIENT_SECRET = os.environ.get('REDDIT_NLP_SECRET')
TARGET_SUBREDDIT_NAME = 'worldnews'

# REPUBLICAN SUBREDDITS LIST :  r/conservative r/republican 

In [20]:
reddit= praw.Reddit(user_agent='Comment Extraction (by u/Reddit_nlp_pa)',
                    client_id=CLIENT_ID, client_secret=CLIENT_SECRET,
                    username=USERNAME, password=PASSWORD)
# max_results_per_request necessary due to bug
api = PushshiftAPI(reddit,max_results_per_request=100)

In [21]:
#testing praw
subreddit = reddit.subreddit(TARGET_SUBREDDIT_NAME)
print(subreddit.display_name)  # Output: worldnews

worldnews


In [34]:
POST_FILENAME = 'filtered_republican_posts.csv'
filtered_posts = pd.read_csv(POST_FILENAME)

In [35]:
fullnames = list(filtered_posts['fullname'])


In [36]:
len(fullnames)

2142

In [37]:
#fetching all comments from every post in the filtered list

OUTPUT_FILENAME = 'republican_comments_raw.csv'

comment_f_start = time.time()

subreddit_comments = []
index =0
for post in reddit.info(fullnames):
    current_comments = post.comments
    current_comments.replace_more(limit=None)
    current_comments_list = current_comments.list()
    
    #merging current comments with master comment list
    subreddit_comments += current_comments_list
    index += 1
    if index % 100 == 0:
        current_time = time.time()
        print(f'{index} posts parsed. Elapsed time: {current_time-comment_f_start}')
        print(f'\tCurrent comment count: {len(subreddit_comments)}')
        
        #making backup
        raw_comments_df = pd.DataFrame([[c.author, c.body, c.score, c.subreddit, c.created_utc, c.id, c.submission.title, c.submission.id]  for c in subreddit_comments])
        raw_comments_df.rename(columns={0: 'author', 1: 'body', 2: 'score', 3: 'subreddit', 4: 'created', 5: 'id', 6: 'post', 7: 'post_title'})
        raw_comments_df.to_csv(OUTPUT_FILENAME, index=False)
    

# saving at the end
raw_comments_df = pd.DataFrame([[c.author, c.body, c.score, c.subreddit, c.created_utc, c.id, c.submission.title, c.submission.id]  for c in subreddit_comments])
raw_comments_df.rename(columns={0: 'author', 1: 'body', 2: 'score', 3: 'subreddit', 4: 'created', 5: 'id', 6: 'post', 7: 'post_title'})
raw_comments_df.to_csv(OUTPUT_FILENAME, index=False)      
        

        
    

100 posts parsed. Elapsed time: 277.01581835746765
	Current comment count: 17652
200 posts parsed. Elapsed time: 619.0872783660889
	Current comment count: 37429
300 posts parsed. Elapsed time: 977.0248873233795
	Current comment count: 57148
400 posts parsed. Elapsed time: 1334.4629926681519
	Current comment count: 77683
500 posts parsed. Elapsed time: 1807.171565771103
	Current comment count: 99635
600 posts parsed. Elapsed time: 2155.6665120124817
	Current comment count: 119997
700 posts parsed. Elapsed time: 2681.137781381607
	Current comment count: 142278
800 posts parsed. Elapsed time: 3025.9003767967224
	Current comment count: 161540
900 posts parsed. Elapsed time: 3350.7395780086517
	Current comment count: 182519
1000 posts parsed. Elapsed time: 3631.049623966217
	Current comment count: 202594
1100 posts parsed. Elapsed time: 3977.894493818283
	Current comment count: 222950
1200 posts parsed. Elapsed time: 4332.379896640778
	Current comment count: 243335
1300 posts parsed. Elapse